In [ ]:
import intake
from cartopy import crs as ccrs
import hvplot.xarray
import holoviews as hv
import geoviews as gv
from geoviews import tile_sources as gvts
import panel as pn
import geoviews.feature as gf
from geoviews import opts
from geoviews import tile_sources as gvts
colormaps = hv.plotting.list_cmaps()

In [ ]:
master_catalog = 'https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml'
cat = intake.open_catalog(master_catalog)

In [ ]:
ocean = cat.ocean
hydro = cat.hydro
atmos = cat.atmosphere

In [ ]:
gui = intake.gui 
gui.add(master_catalog)
gui

Create plot for these datasets 

In [ ]:
# ds = gui.item  #cmip6.A3hr.BCC_CSM2_MR.historical.r1i1p1f1.zos
# ds = gui.item #cmip6.A3hr.CNRM_CM6_1.historical.r1i1p1f2.huss
# ds = ocean.sea_surface_height
ds = atmos.wrf50_erai

In [ ]:
ds = ds.to_dask()

### Template Code

In [ ]:
vars = []
for var in ds.data_vars:
    vars.append(var)

crs = ccrs.PlateCarree()
var_select = pn.widgets.Select(name='Variables:', options=vars, value=vars[0])
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)  
cmap_select = pn.widgets.Select(name='Color Map:', options=colormaps, value='jet')

def plot(var=None, base_map=None,cmap = None):
    base_map = base_map or base_map_select.value
    var = var or var_select.value
    cmap = cmap or cmap_select.value
    mesh = ds[var][-2:,:,:].hvplot.quadmesh(rasterize=True, title=var,datashade = True,
                                    width=600, height=400, rot = 60, dynamic = True,
                                    groupby=list(ds[var].dims[:-2]),cmap = cmap)
    mesh = mesh.redim.default(**{d: ds[d].values.max() for d in ds[var].dims[:-2]})
    overlay = (base_map * mesh.opts(alpha=0.5)).opts(active_tools=['wheel_zoom', 'pan'])
    widgets = {dim: pn.widgets.Select for dim in ds[var].dims[:-2]}
    return pn.pane.HoloViews(overlay, widgets=widgets).layout

def on_var_select(event):
    var = event.obj.value
    dashboard[-1] = plot(var=var)
    
def on_base_map_select(event):
    base_map = event.obj.value
    dashboard[-1] = plot(base_map=base_map)

def on_cmap_select(event):
    cmap = event.obj.value
    dashboard[-1] = plot(cmap=cmap)
    
    
var_select.param.watch(on_var_select, parameter_names=['value']);
base_map_select.param.watch(on_base_map_select, parameter_names=['value']);
cmap_select.param.watch(on_cmap_select, parameter_names=['value']);

dashboard = pn.Column(var_select, base_map_select, cmap_select,plot(var_select.value))

In [ ]:
dashboard.servable()